In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
%aimport bitcoin
import torch
import pyro
import time
import pyro.distributions as dist
import pyro.optim as optim
import numpy as np
import logging
from torch.distributions import constraints
from bitcoin import BitcoinOTC
from pyro.infer import SVI, TraceGraph_ELBO

# Validation checks
logging.basicConfig(format='%(message)s', level=logging.INFO)
pyro.enable_validation(True)

In [3]:
data = bitcoin.BitcoinOTC()

c:\users\andrew\anaconda3\envs\ppp\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\users\andrew\anaconda3\envs\ppp\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\users\andrew\anaconda3\envs\ppp\lib\site-packages\numpy\core\_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
c:\users\andrew\anaconda3\envs\ppp\lib\site-packages\numpy\core\_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
c:\users\andrew\anaconda3\envs\ppp\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [4]:
omega_scale = 1.0
obs_scale = 1.0
embed_dim = 2

def guide(data, node_ind, edge_ind, edge_list):
    r"""Defines a variational family to use to fit an approximate posterior
    distribution for the probability model defined in model.
    """
    # Deleting arguments not used in the guide for linting purposes
    del edge_ind, edge_list
    
    # Parameters governing the priors on the embedding vectors
    omega_loc = pyro.param('omega_loc', lambda: 0.5*torch.randn(embed_dim, data.num_nodes))
    omega_scale = pyro.param('omega_scale', torch.tensor(1.0), constraint=constraints.positive)

    # Parameters governing the prior for the linear regression
    beta_loc = pyro.param('beta_loc', 0.5*torch.randn(embed_dim))
    beta_scale = pyro.param('beta_scale', torch.tensor(1.0), constraint=constraints.positive)
    mu_loc = pyro.param('mu_loc', torch.randn(1))
    mu_scale = pyro.param('mu_scale', torch.tensor(1.0), constraint=constraints.positive)

    # Sample the coefficient vector and intercept for linear regression
    beta = pyro.sample('beta', dist.MultivariateNormal(
        loc=beta_loc, covariance_matrix=(beta_scale**2)*torch.eye(embed_dim)
    ))
    mu = pyro.sample('mu', dist.Normal(mu_loc, mu_scale))

    # (Sub)sample embedding vectors
    with pyro.plate('nodes', size=data.num_nodes, subsample=node_ind):
        omega = pyro.sample('omega', dist.Normal(loc=omega_loc, scale=omega_scale*torch.ones(embed_dim, data.num_nodes))).index_select(-1, node_ind)

    # Define plate for the edge subsampling to pass to model object
    # with pyro.plate('edges', size=data.num_edges, subsample=subsample['edge_ind']):
        # Note: we use pyro plate here as we need to keep the subsampling
        # persistent in both our model and guide functions, we so we
        # define an empty plate here just so we can use the same call
        # in the model.
        # pass

    return omega, beta, mu


def model(data, node_ind, edge_ind, edge_list):
    r"""Defines a probabilistic model for the observed network data."""
    # Define priors on the regression coefficients
    mu = pyro.sample('mu', dist.Normal(torch.tensor(0.0), torch.tensor(2.0)))
    beta = pyro.sample('beta', dist.MultivariateNormal(
        loc=torch.zeros(embed_dim), covariance_matrix=4*torch.eye(embed_dim)
    ))

    # Define prior on the embedding vectors, do subsampling for the
    # embedding vector and then the likelihood term for the observed nodes
    with pyro.plate('nodes', size=data.num_nodes):
        # Embedding vectors of shape [embed_dim, data.num_nodes]
        omega = pyro.sample('omega', dist.Normal(loc=torch.zeros(embed_dim, data.num_nodes), scale=omega_scale*torch.eye(embed_dim, data.num_nodes))).index_select(-1, node_ind)

        # Draw Bernoulli, with or without data depending on if it is observed
        logit_prob = mu + torch.mv(omega.t(), beta)
        
        # Create mask corresponding to entries of ind which lie within data.nodes_train
        obs_mask = np.isin(ind_nodes.numpy(), data.nodes_train.numpy()).tolist()
        valid_data = data.gt[ind].clone()
        valid_data[~obs_mask] = 0
        with poutine.mask(mask=obs_mask):
            pyro.sample('trust', dist.Bernoulli(logits=logit_prob), obs=valid_data)

    # Begin extracting the relevant components of the gram matrix formed by omega now
    # Note that to extract the relevant indices, we need to account for the
    # change in indexing induced by subsampling omega
    gram = torch.mm(omega.t(), omega)
    t = torch.zeros(node_ind.max() + 1, dtype=torch.long)
    t[node_ind] = torch.arange(len(node_ind))
    gram_sample = gram[t[edge_list[0, :]], t[edge_list[0, :]]]
        
    # Draw terms corresponding to the edges
    with pyro.plate('edges', size=data.num_edges, subsample=edge_ind):
        pyro.sample('a', dist.Normal(loc=gram_sample, scale=obs_scale*torch.ones(len(edge_ind))), obs=data.edge_weight_logit[edge_ind])
        

# Define SVI object
svi = SVI(model,
          guide,
          optim.Adam({"lr": .05}),
          loss=TraceGraph_ELBO())

In [5]:
t0 = time.time()

pyro.clear_param_store()
num_iters = 200
sample_args_dict = {'sample_str': 'p-sampling', 'sample_args': {'sample_prob': 0.005}}

for i in range(num_iters):
    subsample_dict = data.subsample(**sample_args_dict)
    elbo = svi.step(data, **subsample_dict)
    if i % 5 == 0:
        logging.info("Elbo loss: {}".format(elbo))
        if (i > 0):
            t1 = time.time()
            logging.info("Time to run 5 gradient steps of SVI: {}".format(t1-t0))
            t0 = time.time()

ValueError: Shape mismatch inside plate('nodes') at site omega dim -1, 278 vs 6005
Trace Shapes:         
 Param Sites:         
    omega_loc 2 6005  
  omega_scale         
     beta_loc      2  
   beta_scale         
       mu_loc      1  
     mu_scale         
Sample Sites:         
    beta dist      | 2
        value      | 2
      mu dist 1    |  
        value 1    |  